In [1]:
import os

list_file = sorted(list(os.walk('./file/1.item_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[1]

file

'crawler_2.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/1.item_number/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Item Number'].tolist()

总数量：4530


['301284845158',
 '391525907146',
 '390952552830',
 '371556118150',
 '301357432588',
 '391525907148',
 '371165049724',
 '371570167825',
 '301791991081',
 '373222887187',
 '302604677927',
 '371434593545',
 '393451025786',
 '394980660785',
 '391247775443',
 '373851006129',
 '371166945093',
 '305529163400',
 '372039504445',
 '392419418698',
 '371587686598',
 '301431221528',
 '394579402606',
 '304605441478',
 '301431218105',
 '304524340437',
 '371209686647',
 '393433942721',
 '303625801667',
 '374721485764',
 '375493102861',
 '375031736986',
 '395263429653',
 '305783436433',
 '304948201971',
 '375645754571',
 '391822141369',
 '394513219346',
 '391894524076',
 '394963935914',
 '371434593548',
 '371891164052',
 '372670598287',
 '373608845817',
 '375449213031',
 '372051901980',
 '372451277683',
 '372450127221',
 '371209484673',
 '391944680997',
 '391712134947',
 '395683815153',
 '395364509835',
 '304010539354',
 '304902753779',
 '390924371121',
 '394481070564',
 '371209702981',
 '374744202617

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = f'''https://www.ebay.com/itm/{input_.loc[a, 'Item Number']}?_ul=US&_stpos=91710&orig_cvip=true'''
        
        resp = requests.get(request_url,
                            headers=get_header(ua=False),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200 or resp.status_code == 404:
            break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
resp.status_code == 404

False

In [9]:
import json

dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

dict_

{'o': {'l': 1,
  'g': {'enableRumWeb': False,
   'permanentSignal': True,
   'disableUnsafeInline': False,
   'enableATCRedesign': 'false',
   'lazyLoaderType': 'NATIVE',
   'largerImageSize': 'FIXED_15',
   'signedIn': False,
   'epInfo': {'featureFlagDetails': {'enableRumWeb': {'value': False},
     'dwebDeliSSE': {'value': {'NORI_ENABLED': 'true'}, 'dropTag': 'server'},
     'dwebAtfStreamingFeature': {'value': {'NORI_ENABLED': 'false',
       'DELI_ENABLED': 'true'},
      'dropTag': 'server'},
     'prefetchResourceFlag': {'value': {'VINODE_ENABLE_PREFETCH_RESOURCES': 'false'}},
     'dwebAtfSplitFeature': {'value': None},
     'enableOTPFeature': {'value': {'enableOTPFlow': 'false',
       'enableGuestOTPAuthFlow': False},
      'dropTag': 'browser'},
     'enableATCRedesign': {'value': {'enableATCLayerV3': 'false'}},
     'disableUnsafeInline': {'value': False},
     'enableAIImageDisclosure': {'value': False},
     'lazyLoaderType': {'value': False},
     'enableAdBlockerDetect

In [18]:
dict_['o']['g']['itemId'].strip() != input_.loc[a, 'Item Number']

False

In [10]:
index_model = -1
for i in range(len(dict_['o']['w'])):
    if 'model' in dict_['o']['w'][i][2]:
        index_model = i
        break

index_model

0

In [11]:
index_model == -1

False

In [12]:
from bs4 import BeautifulSoup
from lxml import etree

for data in dict_['o']['w'][index_model][2]['model']['modules']['MARS']['metaData']:
    if 'hreflang="x-default"' in data:
        soup = BeautifulSoup(data, 'lxml')
        html = etree.HTML(str(soup))

        url = html.xpath('//link/@href')[0]

        if url.startswith('https://www.ebay.com/'):
            url = f'{url}?_ul=US&_stpos=91710&orig_cvip=true'
        elif url.startswith('https://www.ebay.de/'):
            url = f'{url}?_ul=DE&_stpos=10115&orig_cvip=true'
        elif url.startswith('https://www.ebay.co.uk/'):
            url = f'{url}?_ul=GB&_stpos=BS81QU&orig_cvip=true'
        elif url.startswith('https://www.ebay.com.au/'):
            url = f'{url}?_ul=AU&_stpos=2019&orig_cvip=true'
        elif url.startswith('https://www.ebay.ca/'):
            url = f'{url}?_ul=CA&_stpos=M5S2E8&orig_cvip=true'
        else:
            print('fuck')

url

'https://www.ebay.de/itm/301284845158?_ul=DE&_stpos=10115&orig_cvip=true'

In [13]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Item Number': input_.loc[a, 'Item Number'],
                         'Url': url}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Item Number,Url
0,1,301284845158,https://www.ebay.de/itm/301284845158?_ul=DE&_s...


In [14]:
crawler_status = 'ok'

crawler_status

'ok'

In [15]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [16]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_url.xlsx', index=False)

output_correct

,No,Item Number,Url
0,1,301284845158,https://www.ebay.de/itm/301284845158?_ul=DE&_s...


In [17]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_url_error.xlsx', index=False)

output_error

""
